<a href="https://colab.research.google.com/github/dinoanasta/IsiZulu-To-English/blob/main/IsiZulu-To-English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [2]:
import collections
from collections import defaultdict

import helper
import numpy as np
import pandas as pd
import math
import os
import cv2
import random
import skimage
from skimage.color import rgb2gray
from skimage import img_as_float
import matplotlib.pyplot as plt
import scipy.stats as ss
from sklearn.model_selection import KFold

from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional, Dropout, LSTM
from keras.layers.embeddings import Embedding
#from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

from tokenizers import Tokenizer as Toke
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace


In [3]:
from google.colab import drive

In [6]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
pwd

'/content'

In [8]:
cd drive/MyDrive/NLP Project

/content/drive/MyDrive/NLP Project


In [9]:
# read in images
file_names = os.listdir()
print(file_names)
print(len(file_names))

['COMS4054A _ COMS7066A - NLP - Project.pdf', 'en-zu.training.csv', 'zu-en.eval.csv', 'zu-en.training.csv', 'en-zu.eval.csv', 'Data Statement for Umsuka isiZulu Parallel Corpus (1).pdf']
6


In [33]:
df_train = pd.read_csv('en-zu.training.csv')
print(df_train.head(3).columns)
df_eval = pd.read_csv('en-zu.eval.csv')
print(df_eval.head(3).columns)

Index(['en', 'zu', 'source'], dtype='object')
Index(['en', 'zu', 'zu.1', 'Source'], dtype='object')


In [34]:
print(df_train['en'], df_train['zu'])
english_sentences = df_train['en']
zulu_sentences = df_train['zu']

for i in range(len(english_sentences)):
  english_sentences[i] = english_sentences[i].lower()
  zulu_sentences[i] = zulu_sentences[i].lower()

0       His solo albums in the 60s were some of the mo...
1       From which it follows that if you want to save...
2       Thirty years after St. John Paul II begged Moz...
3       Wynford yw sylfaenydd Stafell Fyw yng Nghaerdy...
4       Unai Emery's side have won just two of their l...
                              ...                        
4734    The picture shows the infant breastfeeding whi...
4735    Blame the producers, who constantly switch and...
4736                              (Javier Garcia/BPI/REX)
4737    "I have enormous respect for Laura, She does a...
4738    But despite the chaotic circumstances of the s...
Name: en, Length: 4739, dtype: object 0       Ama-albhamu akhe e-solo eminyaka yawo-60 angam...
1       Ukusuka lapho kuzobe sekulandela ukuthi uma uf...
2       Ngemuva kweminyaka engamashumi amathathu u-St....
3       Wynford yw sylfaenydd Stafell Fyw yng Nghaerdy...
4       Iqembu lika-Unai Emery lisanqobe imidlalo emib...
                              ... 

In [35]:
# BPETokenizer training
english_vocab_size_bpe = 10000
zulu_vocab_size_bpe = 10000

tokenizer_en = Toke(BPE(unk_token="[UNK]"))
tokenizer_zu = Toke(BPE(unk_token="[UNK]"))

trainer_en = BpeTrainer(vocab_size = english_vocab_size_bpe, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
trainer_zu = BpeTrainer(vocab_size = zulu_vocab_size_bpe, special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])

tokenizer_en.pre_tokenizer = Whitespace()
tokenizer_zu.pre_tokenizer = Whitespace()

#files = [f"data/wikitext-103-raw/wiki.{split}.raw" for split in ["test", "train", "valid"]]
tokenizer_en.train_from_iterator(english_sentences, trainer_en)
tokenizer_zu.train_from_iterator(zulu_sentences, trainer_zu)

output = tokenizer_en.encode(("Hello, y'all! How are you 😁 ?").lower())
output1 = tokenizer_en.encode(("abukho ubuholi obungasusa abaphikisana nama-semites.").lower())
print(output.tokens)
print(output1.tokens)
outputz = tokenizer_zu.encode(("Hello, y'all! How are you 😁 ?").lower())
outputz1 = tokenizer_zu.encode(("abukho ubuholi obungasusa abaphikisana nama-semites.").lower())
print(outputz.tokens)
print(outputz1.tokens)
print(tokenizer_en.token_to_id("hel"))
print(tokenizer_en.id_to_token(1))

['hello', ',', 'y', "'", 'all', '!', 'how', 'are', 'you', '[UNK]', '?']
['ab', 'uk', 'ho', 'u', 'bu', 'hol', 'i', 'ob', 'un', 'gas', 'usa', 'ab', 'ap', 'hik', 'is', 'ana', 'nam', 'a', '-', 'semit', 'es', '.']
['hel', 'lo', ',', 'y', "'", 'all', '!', 'ho', 'w', 'are', 'you', '[UNK]', '?']
['abukho', 'ubuholi', 'obunga', 'susa', 'abaphikisana', 'nama', '-', 'semi', 'tes', '.']
511
[CLS]


In [36]:
# Remove empty rows and maybe rows that have sequences of length longer than x
length_counts_en = defaultdict(lambda: 0)
length_counts_zu = defaultdict(lambda: 0)
inds_to_remove = []
for i in range(len(english_sentences)):
  length_counts_en[len(english_sentences[i].split())] += 1
  length_counts_zu[len(zulu_sentences[i].split())] += 1
  if len(english_sentences[i].split()) > 20 or len(zulu_sentences[i].split()) > 20:
    inds_to_remove.append(i)

old_eng_sent = english_sentences
old_zu_sent = zulu_sentences
english_sentences = english_sentences.drop(inds_to_remove, axis=0)
zulu_sentences = zulu_sentences.drop(inds_to_remove, axis=0)

english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
zulu_words_counter = collections.Counter([word for sentence in zulu_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} Zulu words.'.format(len([word for sentence in zulu_sentences for word in sentence.split()])))
print('{} unique Zulu words.'.format(len(zulu_words_counter)))
print('10 Most common words in the Zulu dataset:')
print('"' + '" "'.join(list(zip(*zulu_words_counter.most_common(10)))[0]) + '"')

27271 English words.
9538 unique English words.
10 Most common words in the English dataset:
"the" "to" "a" "and" "in" "of" "is" "for" "was" "on"

21288 Zulu words.
14241 unique Zulu words.
10 Most common words in the Zulu dataset:
"ukuthi" "futhi" "kakhulu" "noma" "kodwa" "uma" "kusho" "kanye" "-" "kube"


In [37]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x)
    return tokenizer.texts_to_sequences(x), tokenizer

def bpe_tokenize(x, tokenizer_to_use):
  # return the bpe tokenization of each sentence as a sequence
  all_seq = []
  for curr_sent in x:
    otp = tokenizer_to_use.encode(curr_sent)
    curr_tokenization = otp.tokens
    curr_seq = []
    for curr_token in curr_tokenization:
      curr_seq.append(tokenizer_to_use.token_to_id(curr_token))

    all_seq.append(curr_seq)
  
  return all_seq

In [38]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    return pad_sequences(x, maxlen=length, padding='post')

In [39]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x_bpe = bpe_tokenize(x, tokenizer_en)
    preprocess_y_bpe = bpe_tokenize(y, tokenizer_zu)

    #Print min and max values of zulu sentences
    min_zu = 2000
    max_zu = 0
    for sample_i, (sent, token_sent) in enumerate(zip(y, preprocess_y_bpe)):
      if len(token_sent) != 0:
        min_zu = min(min_zu, min(token_sent))
        max_zu = max(max_zu, max(token_sent))
    print('Min:', min_zu, ' Max:', max_zu)


    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    preprocess_x_bpe = pad(preprocess_x_bpe)
    preprocess_y_bpe = pad(preprocess_y_bpe)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)
    preprocess_y_bpe = preprocess_y_bpe.reshape(*preprocess_y_bpe.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk, preprocess_x_bpe, preprocess_y_bpe

preproc_english_sentences, preproc_zulu_sentences, english_tokenizer, zulu_tokenizer, preproc_english_sentences_bpe, preproc_zulu_sentences_bpe =\
    preprocess(english_sentences, zulu_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_zulu_sequence_length = preproc_zulu_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
zulu_vocab_size = len(zulu_tokenizer.word_index)

max_english_sequence_length_bpe = preproc_english_sentences_bpe.shape[1]
max_zulu_sequence_length_bpe = preproc_zulu_sentences_bpe.shape[1]

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max Zulu sentence length:", max_zulu_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("Zulu vocabulary size:", zulu_vocab_size)

print()
print("Max English sentence length(bpe):", max_english_sequence_length_bpe)
print("Max Zulu sentence length(bpe):", max_zulu_sequence_length_bpe)
print("English vocabulary size(bpe):", english_vocab_size_bpe)
print("Zulu vocabulary size(bpe):", zulu_vocab_size_bpe)

Min: 5  Max: 9998
Data Preprocessed
Max English sentence length: 23
Max Zulu sentence length: 25
English vocabulary size: 7639
Zulu vocabulary size: 12597

Max English sentence length(bpe): 42
Max Zulu sentence length(bpe): 58
English vocabulary size(bpe): 10000
Zulu vocabulary size(bpe): 10000


In [40]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    #print("ITW:", index_to_words)
    #print("ITW[42]:", index_to_words[42])
    print(logits, np.shape(logits))
    #print([index_to_words[prediction[0]] for prediction in logits])
    print([prediction for prediction in np.argmax(logits, 1)])

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])
    #return ' '.join([index_to_words[prediction[0]] for prediction in logits])


print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


In [41]:
def logits_to_text_bpe(logits, tokenizer):
  #return ' '.join([tokenizer.id_to_token(prediction) for prediction in logits])
  return ' '.join([tokenizer.id_to_token(prediction) for prediction in np.argmax(logits, 1)])

#   print(preproc_english_sentences_bpe[0])
# print(logits_to_text_bpe(preproc_english_sentences_bpe[0], tokenizer_en))

In [42]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(GRU(256, input_shape=input_shape[1:], return_sequences=True))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size+1, activation='softmax'))) 
    #model.add(TimeDistributed(Dense(2500, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate),
                  metrics=['accuracy'])
    return model



In [43]:
#tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_zulu_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_zulu_sentences.shape[-2], 1))

#print('Tmp_x', tmp_x[0], np.shape(tmp_x))
#print(logits_to_text(tmp_x[0], english_tokenizer))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_zulu_sequence_length,
    english_vocab_size,
    zulu_vocab_size)

print(simple_rnn_model.summary())

simple_rnn_model.fit(tmp_x, preproc_zulu_sentences, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], zulu_tokenizer))

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_9 (GRU)                 (None, 25, 256)           198912    
                                                                 
 time_distributed_16 (TimeDi  (None, 25, 1024)         263168    
 stributed)                                                      
                                                                 
 dropout_8 (Dropout)         (None, 25, 1024)          0         
                                                                 
 time_distributed_17 (TimeDi  (None, 25, 12598)        12912950  
 stributed)                                                      
                                                                 
Total params: 13,375,030
Trainable params: 13,375,030
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
2/2 [=====================

In [44]:
#tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x_bpe = pad(preproc_english_sentences_bpe, max_zulu_sequence_length_bpe)
tmp_x_bpe = tmp_x_bpe.reshape((-1, preproc_zulu_sentences_bpe.shape[-2], 1))

#print('Tmp_x', tmp_x[0], np.shape(tmp_x))
#print(logits_to_text(tmp_x[0], english_tokenizer))

# Train the neural network
simple_rnn_model_bpe = simple_model(
    tmp_x_bpe.shape,
    max_zulu_sequence_length_bpe,
    english_vocab_size_bpe,
    zulu_vocab_size_bpe)

print(simple_rnn_model_bpe.summary())

simple_rnn_model_bpe.fit(tmp_x_bpe, preproc_zulu_sentences_bpe, batch_size=1024, epochs=10, validation_split=0.2)

# Print prediction(s)
print(logits_to_text_bpe(simple_rnn_model_bpe.predict(tmp_x_bpe[:1])[0], tokenizer_zu))

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_10 (GRU)                (None, 58, 256)           198912    
                                                                 
 time_distributed_18 (TimeDi  (None, 58, 1024)         263168    
 stributed)                                                      
                                                                 
 dropout_9 (Dropout)         (None, 58, 1024)          0         
                                                                 
 time_distributed_19 (TimeDi  (None, 58, 10001)        10251025  
 stributed)                                                      
                                                                 
Total params: 10,713,105
Trainable params: 10,713,105
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
2/2 [=====================

In [45]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(GRU(256, return_sequences=True))    
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate),
                  metrics=['accuracy'])
    return model

#tests.test_embed_model(embed_model)

# TODO: Reshape the input


In [46]:
tmp_x = pad(preproc_english_sentences, preproc_zulu_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_zulu_sentences.shape[-2]))

# TODO: Train the neural network
embed_rnn_model = embed_model(
    tmp_x.shape,
    preproc_zulu_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(zulu_tokenizer.word_index)+1)

embed_rnn_model.summary()

embed_rnn_model.fit(tmp_x, preproc_zulu_sentences, batch_size=1024, epochs=200, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(embed_rnn_model.predict(tmp_x[:1])[0], zulu_tokenizer))

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 25, 256)           1955840   
                                                                 
 gru_11 (GRU)                (None, 25, 256)           394752    
                                                                 
 time_distributed_20 (TimeDi  (None, 25, 1024)         263168    
 stributed)                                                      
                                                                 
 dropout_10 (Dropout)        (None, 25, 1024)          0         
                                                                 
 time_distributed_21 (TimeDi  (None, 25, 12598)        12912950  
 stributed)                                                      
                                                                 
Total params: 15,526,710
Trainable params: 15,526,710

In [25]:
tmp_x_bpe = pad(preproc_english_sentences_bpe, preproc_zulu_sentences_bpe.shape[1])
tmp_x_bpe = tmp_x_bpe.reshape((-1, preproc_zulu_sentences_bpe.shape[-2]))

# TODO: Train the neural network
embed_rnn_model_bpe = embed_model(
    tmp_x_bpe.shape,
    preproc_zulu_sentences_bpe.shape[1],
    english_vocab_size_bpe+1,
    zulu_vocab_size_bpe+1)

embed_rnn_model_bpe.summary()

embed_rnn_model_bpe.fit(tmp_x_bpe, preproc_zulu_sentences_bpe, batch_size=1024, epochs=200, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text_bpe(embed_rnn_model_bpe.predict(tmp_x_bpe[:1])[0], tokenizer_zu))

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 64, 256)           1280256   
                                                                 
 gru_4 (GRU)                 (None, 64, 256)           394752    
                                                                 
 time_distributed_8 (TimeDis  (None, 64, 1024)         263168    
 tributed)                                                       
                                                                 
 dropout_4 (Dropout)         (None, 64, 1024)          0         
                                                                 
 time_distributed_9 (TimeDis  (None, 64, 5001)         5126025   
 tributed)                                                       
                                                                 
Total params: 7,064,201
Trainable params: 7,064,201
No

In [28]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.003
    
    # TODO: Build the layers
    model = Sequential()
    model.add(Embedding(english_vocab_size, 256, input_length=input_shape[1], input_shape=input_shape[1:]))
    model.add(Bidirectional(GRU(128, return_sequences=True), input_shape=input_shape[1:]))
    model.add(TimeDistributed(Dense(1024, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax'))) 

    # Compile model
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate),
                  metrics=['accuracy'])
    return model



In [29]:
#tests.test_bd_model(bd_model)

# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, preproc_zulu_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_zulu_sentences.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model1 = bd_model(
    tmp_x.shape,
    preproc_zulu_sentences.shape[1],
    english_vocab_size+1,
    zulu_vocab_size+1)

embed_rnn_model1.summary()

embed_rnn_model1.fit(tmp_x, preproc_zulu_sentences, batch_size=1024, epochs=50, validation_split=0.2)

print(logits_to_text(embed_rnn_model1.predict(tmp_x[:1])[0], zulu_tokenizer))

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 22, 256)           1315072   
                                                                 
 bidirectional (Bidirectiona  (None, 22, 256)          296448    
 l)                                                              
                                                                 
 time_distributed_10 (TimeDi  (None, 22, 1024)         263168    
 stributed)                                                      
                                                                 
 dropout_5 (Dropout)         (None, 22, 1024)          0         
                                                                 
 time_distributed_11 (TimeDi  (None, 22, 7841)         8037025   
 stributed)                                                      
                                                      

In [50]:
tmp_x_bpe = pad(preproc_english_sentences_bpe, preproc_zulu_sentences_bpe.shape[1])
tmp_x_bpe = tmp_x_bpe.reshape((-1, preproc_zulu_sentences_bpe.shape[-2]))

# TODO: Train and Print prediction(s)
embed_rnn_model1_bpe = bd_model(
    tmp_x_bpe.shape,
    preproc_zulu_sentences_bpe.shape[1],
    english_vocab_size_bpe+1,
    zulu_vocab_size_bpe+1)

embed_rnn_model1_bpe.summary()

embed_rnn_model1_bpe.fit(tmp_x_bpe, preproc_zulu_sentences_bpe, batch_size=1024, epochs=20, validation_split=0.2)

print(logits_to_text_bpe(embed_rnn_model1_bpe.predict(tmp_x_bpe[:1])[0], tokenizer_zu))

Model: "sequential_14"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 58, 256)           2560256   
                                                                 
 bidirectional_7 (Bidirectio  (None, 58, 256)          296448    
 nal)                                                            
                                                                 
 time_distributed_28 (TimeDi  (None, 58, 1024)         263168    
 stributed)                                                      
                                                                 
 dropout_14 (Dropout)        (None, 58, 1024)          0         
                                                                 
 time_distributed_29 (TimeDi  (None, 58, 10001)        10251025  
 stributed)                                                      
                                                     

In [31]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement

    # Hyperparameters
    learning_rate = 0.005
    
    # Build the layers    
    model = Sequential()
    # Embedding
    model.add(Embedding(english_vocab_size, 128, input_length=input_shape[1],
                         input_shape=input_shape[1:]))
    # Encoder
    model.add(Bidirectional(GRU(128)))
    model.add(RepeatVector(output_sequence_length))
    # Decoder
    model.add(Bidirectional(GRU(128, return_sequences=True)))
    model.add(TimeDistributed(Dense(512, activation='relu')))
    model.add(Dropout(0.5))
    model.add(TimeDistributed(Dense(french_vocab_size, activation='softmax')))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate),
                  metrics=['accuracy'])
    return model

#tests.test_model_final(model_final)

print('Final Model Loaded')

Final Model Loaded


In [32]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = model_final(x.shape,y.shape[1],
                        len(x_tk.word_index)+1,
                        len(y_tk.word_index)+1)
    model.summary()
    model.fit(x, y, batch_size=1024, epochs=200, validation_split=0.2)

    
    print(logits_to_text(model.predict(x[:1])[0], zulu_tokenizer))
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    # y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    # y_id_to_word[0] = '<PAD>'

    # sentence = 'he saw a old yellow truck'
    # sentence = [x_tk.word_index[word] for word in sentence.split()]
    # sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    # sentences = np.array([sentence[0], x[0]])
    # predictions = model.predict(sentences, len(sentences))

    # print('Sample 1:')
    # print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    # print('Il a vu un vieux camion jaune')
    # print('Sample 2:')
    # print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    # print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_zulu_sentences, english_tokenizer, zulu_tokenizer)

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 19, 128)           657536    
                                                                 
 bidirectional_2 (Bidirectio  (None, 256)              198144    
 nal)                                                            
                                                                 
 repeat_vector (RepeatVector  (None, 22, 256)          0         
 )                                                               
                                                                 
 bidirectional_3 (Bidirectio  (None, 22, 256)          296448    
 nal)                                                            
                                                                 
 time_distributed_14 (TimeDi  (None, 22, 512)          131584    
 stributed)                                           